In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to the database you built
conn = sqlite3.connect('fleet.db')

# Load the recovery actions
df = pd.read_sql_query("SELECT * FROM recovery_actions", conn)
print(f"Loaded {len(df)} processed records.")
df.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='recovery_action', palette='viridis')
plt.title('Fleet Recovery Action Distribution (Post-Chaos Injection)')
plt.ylabel('Number of Devices')
plt.show()

In [ ]:
query = """
WITH RegionalStats AS (
    SELECT 
        region,
        update_status,
        COUNT(*) as count,
        SUM(COUNT(*)) OVER(PARTITION BY region) as total
    FROM ota_logs
    GROUP BY 1, 2
)
SELECT region, update_status, ROUND((count*1.0/total)*100, 2) as percentage
FROM RegionalStats
"""
stats_df = pd.read_sql_query(query, conn)
pivot_df = stats_df.pivot(index='region', columns='update_status', values='percentage')

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_df, annot=True, cmap='YlOrRd')
plt.title('Regional Update Status Heatmap (%)')
plt.show()